In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn import grid_search
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.grid_search import GridSearchCV

import xgboost as xgb

%matplotlib inline

from sklearn.cross_validation import train_test_split

train = pd.read_csv('act_train.csv')
train.drop('activity_id', axis=1, inplace = True) 
train.drop_duplicates(inplace = True)

test = pd.read_csv('act_test.csv')
people = pd.read_csv('people.csv')

In [2]:
test_ids = test['activity_id']
target = train['outcome']

train.drop('outcome', axis=1, inplace = True) 
test.drop(['activity_id'], axis=1, inplace = True) 

    
## Split off _ from people_id
train['people_id'] = train['people_id'].apply(lambda x: x.split('_')[1])
train['people_id'] = pd.to_numeric(train['people_id']).astype(int)

test['people_id'] = test['people_id'].apply(lambda x: x.split('_')[1])
test['people_id'] = pd.to_numeric(test['people_id']).astype(int)
    
columns = list(train.columns)
columns.remove("date")
    
for col in columns[1:]:
    train[col] = train[col].fillna(-1)
    train[col] = train[col].apply(lambda x: x if x == -1 else x.split(' ')[1])
    train[col] = pd.to_numeric(train[col]).astype(int)
    
    test[col] = test[col].fillna(-1)
    test[col] = test[col].apply(lambda x: x if x == -1 else x.split(' ')[1])
    test[col] = pd.to_numeric(test[col]).astype(int)

In [3]:
people['people_id'] = people['people_id'].apply(lambda x: x.split('_')[1])
people['people_id'] = pd.to_numeric(people['people_id']).astype(int)
    
#  Values in the people df is Booleans and Strings    
columns = list(people.columns)
columns.remove("char_38")
bools = columns[12:]
strings = columns[1:12]
strings.remove("date")
    
for col in bools:
    people[col] = pd.to_numeric(people[col]).astype(int)   
    
for col in strings:
    people[col] = people[col].fillna(-2)
    people[col] = people[col].apply(lambda x: x if x == -2 else x.split(' ')[1])
    people[col] = pd.to_numeric(people[col]).astype(int)

In [4]:
def fix_date_time(df):
    def extract_field(_df, start, stop):
        return _df['date'].map(lambda dt: int(dt[start:stop]))
    df['Year'] = extract_field(df,0,4)
    df['Month'] = extract_field(df,5,7)
    df['Day'] = extract_field(df,8,10)
    return df.drop(['date'], axis = 1)
train = fix_date_time(train)
test = fix_date_time(test)
people = fix_date_time(people)

In [5]:
train = train.merge(people, how='left', on='people_id')
test = test.merge(people, how='left', on='people_id')

train.drop("people_id", axis = 1, inplace= True)
test.drop("people_id", axis = 1, inplace= True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.20, random_state=23)

print('# train: %5d (0s: %5d, 1s: %4d)'%(len(y_train), sum(y_train==0), sum(y_train==1)))
print('# test:  %5d (0s: %5d, 1s: %4d)'%((len(y_test), sum(y_test==0), sum(y_test==1))))

# train: 971102 (0s: 499077, 1s: 472025)
# test:  242776 (0s: 124791, 1s: 117985)


In [8]:
# xgb
clf_xgb = xgb.XGBClassifier(missing=np.nan, max_depth=5,
                        n_estimators=1000, learning_rate=0.01, 
                        subsample=0.5, colsample_bytree=0.9, seed=4242)

# fitting
clf_xgb.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="auc", eval_set=[(X_test, y_test)])

test_proba = clf_xgb.predict_proba(test)
test_preds = test_proba[:,1]

# Format for submission
output = pd.DataFrame({ 'activity_id' : test_ids, 'outcome': test_preds })
output.head()
output.to_csv('xgb.csv', index = False)

Will train until validation_0 error hasn't decreased in 50 rounds.
[0]	validation_0-auc:0.889867
[1]	validation_0-auc:0.903838
[2]	validation_0-auc:0.904292
[3]	validation_0-auc:0.903508
[4]	validation_0-auc:0.902630
[5]	validation_0-auc:0.902491
[6]	validation_0-auc:0.905281
[7]	validation_0-auc:0.904681
[8]	validation_0-auc:0.904430
[9]	validation_0-auc:0.904058
[10]	validation_0-auc:0.903322
[11]	validation_0-auc:0.903392
[12]	validation_0-auc:0.904438
[13]	validation_0-auc:0.905345
[14]	validation_0-auc:0.906612
[15]	validation_0-auc:0.907310
[16]	validation_0-auc:0.907194
[17]	validation_0-auc:0.907045
[18]	validation_0-auc:0.906936
[19]	validation_0-auc:0.907153
[20]	validation_0-auc:0.907643
[21]	validation_0-auc:0.907604
[22]	validation_0-auc:0.908072
[23]	validation_0-auc:0.908008
[24]	validation_0-auc:0.908145
[25]	validation_0-auc:0.908220
[26]	validation_0-auc:0.908243
[27]	validation_0-auc:0.908258
[28]	validation_0-auc:0.908140
[29]	validation_0-auc:0.908138
[30]	validati

NameError: name 'test_proba' is not defined

In [10]:
test_proba = clf_xgb.predict_proba(test)
test_preds = test_proba[:,1]

# Format for submission
output = pd.DataFrame({ 'activity_id' : test_ids, 'outcome': test_preds })
output.head()
output.to_csv('xgb.csv', index = False)

In [24]:
%matplotlib qt
xgb.plot_importance(clf_xgb)

In [18]:
%matplotlib qt
xgb.to_graphviz(clf_xgb)

ImportError: You must install graphviz to plot tree

In [19]:
clfRF = RandomForestClassifier()
clfRF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
proba = clfRF.predict_proba(X_test)
preds = proba[:,1]
score = roc_auc_score(y_test, preds)
print("Area under ROC {0}".format(score))

Area under ROC 0.997539135475


In [21]:
# Test Set Predictions
test_proba = clfRF.predict_proba(test)
test_preds = test_proba[:,1]

# Format for submission
output = pd.DataFrame({ 'activity_id' : test_ids, 'outcome': test_preds })
output.head()
output.to_csv('randomForest.csv', index = False)

In [22]:
plt.hist(target)

(array([ 623868.,       0.,       0.,       0.,       0.,       0.,
              0.,       0.,       0.,  590010.]),
 array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]),
 <a list of 10 Patch objects>)

In [23]:
plt.hist(test_preds)

(array([ 171442.,    8259.,   22087.,       0.,   16025.,   51611.,
          39033.,       0.,   47182.,  143048.]),
 array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]),
 <a list of 10 Patch objects>)